In [61]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("NORD_Task") \
      .getOrCreate() 
sc = spark.sparkContext




hadoop_conf=sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")

S3_BUCKET = 's3-nord-challenge-data'
S3_REGION = 'eu-central-1'
hadoop_conf.set("fs.s3a.endpoint", f"s3.{S3_REGION}.amazonaws.com")



#number of files to process - will be read as input
N = 100

### Note 
In general I can see 2 approaches to load files data:
   - Approach 1.
     - `spark.read.format('binaryFile').option("pathGlobFilter","<path-glob>").load(<s3-bucket>)`. This solution would read all files with metadata into single DataFrame (path, mod time,  length, content)
     - parse content of file in apropriate resulted dataframe transformation
   - The advantage of it is that you receive parallelized DataFrame, content of file would be read in lazy way during processing each file. So in theory on a big enough spark cluster spark should take care of distributing and performance for you. The problem seems to be when you have to work with pretty read milions of files with unknown file size. You may end up huge memory and performance issues. This problem is shown e.g. in [this blog article](https://wrightturn.wordpress.com/2015/07/22/getting-spark-data-from-aws-s3-using-boto-and-pyspark/). Although it's pretty old I did not find any more recent solution to the issue. It also describes second approach.
   - Approach 2.
       - list all objects you're interested files in s3 bucket into some collection (but without parallelizing it)
       - create parallelized dataframe based on the given collection
       - read and process file content as part of transformations
   - The bottleneck might that you have to iterate over millions of files so the size of the collection to be processed (on one node) might be huge. 
   
As I am not able to test on a large set of data and big enough spark cluster which approach is more efficient. I am going to use approach described in [mentioned article](https://wrightturn.wordpress.com/2015/07/22/getting-spark-data-from-aws-s3-using-boto-and-pyspark). However instead of using boto3 for listing all objects I will use [`hadoop.fs.path.getFilesystem.globStatus`](https://stackoverflow.com/a/67050173/2018369) because boto3 [seems to be not the most effective way](https://stackoverflow.com/q/69920805/2018369) to get file list.

I was also considering one more approach, which however I could not find any good way to implement. So my idea was to create a dataframe similar to the one created by `spark.read.format('binaryFile').option("pathGlobFilter","<path-glob>").load(<s3-bucket>)`, but which contain only prefix of file (first 1024 or 2048 bytes). This way we could have a Dataframe(path, mod time,  length, PE headers), we could process the header of file to get all required PE metadata apart from imports/expors and in next step we could load apropriate sections of file to get imports/exports.


In [62]:
clean_path = '/0/*.???'
malware_path = '/1/*.???'



cleanPath = sc._jvm.org.apache.hadoop.fs.Path(f's3a://{S3_BUCKET}{clean_path}')
cFs = cleanPath.getFileSystem(hadoop_conf)
clean_files = cFs.globStatus(cleanPath)

malwarePath = sc._jvm.org.apache.hadoop.fs.Path(f's3a://{S3_BUCKET}{malware_path}')
mFs = malwarePath.getFileSystem(hadoop_conf)
malware_files = mFs.globStatus(malwarePath)

In [63]:
import random
print(len(malware_files))
files_to_process = random.sample(clean_files, int(N/2))+ random.sample(malware_files, int(N/2))
print(len(files_to_process))

malware_files

14652
100


JavaObject id=o46234

In [64]:
# put files into dataFrame

from pyspark.sql.types import StructType,StructField, StringType, IntegerType
schema = StructType([       
    StructField('path', StringType(), True),
    StructField('size', IntegerType(), True),
    StructField('type', StringType(), True)
])
data = [(f.getPath().toUri().getRawPath(), f.getLen(), f.getPath().getName().split('.')[-1]) for f in files_to_process]
# data = [(f.getPath().toUri().getRawPath(), f.getLen(), f.getPath().getName().split('.')[-1]) for f in clean_files]
# make sure we don't have duplicates
filesDF= spark.createDataFrame(data=data, schema = schema).distinct()
filesDF.orderBy("size").show()

+--------------------+-----+----+
|                path| size|type|
+--------------------+-----+----+
|/0/yBew1RCScL6IPp...|    0| dll|
|/0/wFmMWSpwZSs018...| 1556| dll|
|/0/NbO24Jqs5manwz...| 2560| dll|
|/0/BQeyNvzPtZbUo4...| 4096| dll|
|/1/YWBdWtgrl6BxYF...| 4096| exe|
|/0/S6uvChiGpJzXwg...| 4096| dll|
|/1/IMGRg0DkRBudYg...| 5120| dll|
|/1/RwJEa3Fphb0MBE...| 6656| exe|
|/0/jRUjmWjrcMTbRu...| 7680| DLL|
|/0/DaUn6epfUwZBFa...| 9242| dll|
|/0/LplBf5IViRpgCZ...| 9728| dll|
|/0/tchpbDHf32pxYu...|11776| dll|
|/0/w8GrAcrVBSApz7...|13824| dll|
|/0/MMGPxOjrgbNgUb...|13840| dll|
|/0/VRaYgpEEdysoq0...|22528| dll|
|/0/v2Y0FOflJsSt42...|24576| dll|
|/0/tOM2zfZl1crwBT...|24576| exe|
|/0/MvxKp6jVjapuM7...|25600| exe|
|/0/aC0BERV8SPcw5J...|26112| exe|
|/1/4LeBU93eqe3JMy...|26112| exe|
+--------------------+-----+----+
only showing top 20 rows



In [60]:
filesDF.orderBy('size').show(50,False)

+---------------------------------------+----+----+
|path                                   |size|type|
+---------------------------------------+----+----+
|/0/yBew1RCScL6IPpDEjBBHoVhqTPDeLz3V.dll|0   |dll |
|/0/kmpXko3MQvii5MesJhEFwn4j5PNjp0Nh.DLL|0   |DLL |
|/0/fSp98VmWIff9hnv6fdlfo5uwYwczb7nd.exe|0   |exe |
|/0/g6JNoxodcclCON5cDCH7nOLzY4tJEX2D.dll|115 |dll |
|/0/N9qADLiJO4Z2GguKu2p0YRLENVFW1v1N.dll|115 |dll |
|/0/oaraVTbrZMLISeMjnYcdbQlykwWOqqzT.dll|116 |dll |
|/0/6cTLVriV86LLDgSLBJEViSvKwmBMQsh6.dll|116 |dll |
|/0/yDKSqu4Jb9SiLF7cW1g8o0Cd14dG3GNJ.exe|127 |exe |
|/0/T0sKa8Vg5a2vDyscxcHADXHYLTP3eFT2.dll|129 |dll |
|/0/FMA3ScgaMpsXNz2NmgJY4vyiEhImw1QV.dll|130 |dll |
|/0/hcrKj0lynV3fLptQPhcPb340fANniTkg.dll|131 |dll |
|/0/yPixkkYO3Zm8PP8mx1V0n5IXcZJkrTdw.dll|131 |dll |
|/0/emZYnrJcxMtYdBoEIR8EPE7ozfSheVZc.dll|132 |dll |
|/0/bMXiMnwowm739HZP9bTs3xVu5g0KHGms.dll|132 |dll |
|/0/5aer54Fijmdk8HDVm8rBplS1MhNb1Nyo.exe|132 |exe |
|/0/ayZv4xeEzSVI9QFMaH6EzEOp5ewWygND.dll|133 |dll |
|/0/KRbhZWyn

# Database Notes
I was considering SQL and NoSQL (key/value store) to store files info.

### SQL Database - MySQL
For now I decided to use simple solution  - MySQL database. This is pretty simple to replace with different SQL database or even with totally different solution in case of efficiency issues.

The architecture is rather not complicated. All **distinct** file records are processed and stored in one table with following schema
 `path Varchar primary key, size Int, type Varchar, architecture Varchar default NULL, imports Int default NULL, exports Int default NULL, INDEX(size, type));`
At the whole table is loaded into DataFrame. It is substracted from task files DF to ensure already processed files are skipped. And after processing transformed DF is appended to existing table in MySQL.

Although number of files processed can reach (hundred of) millions [MySQL should handle it properly](https://dba.stackexchange.com/questions/20335/can-mysql-reasonably-perform-queries-on-billions-of-rows) with proper indexes. If there are billions of rows in DB we might start [encontering problems](https://stackoverflow.com/questions/38346613/mysql-and-a-table-with-100-millions-of-rows)
In case of performance issue we may think about using different Database type.

### NoSQL solutions
I was considering also NoSQL database which very often perform better in distributed environment and in most cases scale horizontally much easier than classical SQL DB. For this task I consider key/value store as a good solution.

#### Aerospike
Aerospike was considered as it promises high efficiency, distributed (based on shared nothing architecture) database for storing key/value pairs. In commercial version it support pyspark distributed operations, direct import to RDDs etc. So if required it might give very good performance.

#### Redis
Open source, in-memory data store used as a database, cache, streaming engine, and message broker.


### "hybrid" approach
The issue with in-memory key/value store is that it does not provide (by default) persistence of data.
This can be achieved both in Redis and Aerospike of course but not by default.

My idea was to provide hybrid solution in which processed files data is stored in classical SQL database but apart from that it is also imported into key/value store. In that case there is no need to load all existing entries into DataFrame prior to processing new entries just to make sure some files weren't already processed. Instead, I would filter out from `filesDF` entries that exists in key/value store during transformations. As a last steps I would need to save (append) `filesDB` not only to SQL database but also to key/value store




In [65]:
# DB Settings
jdbc_url = 'jdbc:mysql://db/nord_files'
table = "files_info"
username = "root"
password = "password"
driver = "com.mysql.cj.jdbc.Driver"


In [66]:
# get files already processed from database

db_files_info = spark.read.format("jdbc")\
    .options(url =jdbc_url,driver=driver, dbtable=table,user=username,password=password).load().select(["path", "size", "type"])


print(db_files_info.columns)
db_files_info.show()





['path', 'size', 'type']
+----+----+----+
|path|size|type|
+----+----+----+
+----+----+----+



In [67]:
# Remove files that exists in DB from list of files to process
filesDF = filesDF.subtract(db_files_info)
filesDF.show()

+--------------------+------+----+
|                path|  size|type|
+--------------------+------+----+
|/1/G0VT13tsEjuIzj...|352256| exe|
|/0/MMGPxOjrgbNgUb...| 13840| dll|
|/0/TB8honzNnNXPKM...|509248| exe|
|/0/ikES4VIEBUyNDM...| 26324| dll|
|/1/tOysallBEHAUAM...| 94208| exe|
|/0/Bb8cYk8FTQ59jz...|194048| dll|
|/0/BIKElaZJq1tsUZ...|124928| dll|
|/0/yBew1RCScL6IPp...|     0| dll|
|/1/eksDCS3H2m7RY7...|572416| exe|
|/1/ljzLUAWAxOBYjs...|259072| exe|
|/0/BQeyNvzPtZbUo4...|  4096| dll|
|/1/4LeBU93eqe3JMy...| 26112| exe|
|/0/rgA1VwXwJxybiZ...| 63488| dll|
|/1/5S1DzOMiFPhiD4...|450763| exe|
|/1/mlBfSv1nduFzzG...|118784| exe|
|/0/S6uvChiGpJzXwg...|  4096| dll|
|/1/5jhmYTl9GoM8Ew...|224256| exe|
|/1/j3MxwPUnTwKTzE...|384512| exe|
|/0/HKB7N1d2XQS9UL...|103536| exe|
|/0/0cwj8FURlEx0Zq...|202008| dll|
+--------------------+------+----+
only showing top 20 rows



In [68]:
# process files 

from s3file_reader import S3FileReader
from pe_parser import PeParser
#import importlib
#importlib.reload(S3FileReader)

reader = S3FileReader(S3_BUCKET, S3_REGION)

schema = StructType(filesDF.schema.fields+[
    StructField('architecture', StringType(), True),
    StructField('imports', IntegerType(), True),
    StructField('exports',IntegerType(), True)
])
#TODO: This should be fixed
parsed=filesDF.rdd.map(lambda x: (*x, *PeParser(x['path'], reader.get_file_stream(x['path']),x['size']).get_short_meta()))

parsedDF = parsed.toDF(schema)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/pyspark/serializers.py", line 437, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/usr/local/lib/python3.10/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/usr/local/lib/python3.10/site-packages/pyspark/cloudpickle/cloudpickle_fast.py", line 563, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle '_thread.lock' object


PicklingError: Could not serialize object: TypeError: cannot pickle '_thread.lock' object

In [34]:
# Store result to DB
parsedDF = filesDF
parsedDF.write.format('jdbc').options(
    url=jdbc_url, driver=driver,dbtable=table, user=username, password=password
).mode('append').save()

